In [3]:
import importlib
import pathlib as pl
import sqlite3

from tqdm import tqdm

import grid_load.processing as glp

importlib.reload(glp)

conn = sqlite3.connect("local.db")

# files = list(pl.Path("../data").glob("*.csv"))
# for file in tqdm(files, total=len(files)):
#     df = glp.extract_data(file)
#     df = glp.transform_data(df)
#     if "Trafostation" in file.stem:
#         df.to_sql("grid_consumption", conn, if_exists="append", index=False)
#     elif "Wohnung" in file.stem:
#         user_id = int(file.stem.split("_")[0])
#         df["user_id"] = user_id
#         df.to_sql("user_consumption", conn, if_exists="append", index=False)

# conn.close()

In [12]:
import pandas as pd

sql = "Select * from grid_consumption order by ts desc"
df_lite = pd.read_sql(sql, conn)

Timestamp('2022-12-31 23:15:00+0000', tz='UTC')

In [23]:
def household_stress_level_calculation(df_grid: pd.DataFrame) -> tuple[float, float]:
    df_grid = df_grid.copy()
    df_grid["date_fmt"] = pd.to_datetime(df_grid["date_fmt"])
    last_wert_month = df_grid["date_fmt"].iloc[-1].month
    last_wert_year = df_grid["date_fmt"].iloc[-1].year
    current_df = df_grid[
        (df_grid["date_fmt"].apply(lambda x: pd.to_datetime(x).month) == last_wert_month)
        & (df_grid["date_fmt"].apply(lambda x: pd.to_datetime(x).year) == last_wert_year)
    ]

    max_value_of_the_month = current_df["Wert"].max()
    household_stress_level = max_value_of_the_month - df_grid["Wert"].iloc[-1]
    return max_value_of_the_month, household_stress_level

household_stress_level_calculation(df_lite.tail())

C:\Users\LuuQ\AppData\Local\Temp\ipykernel_2756\923326423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grid["date_fmt"] = pd.to_datetime(df_grid["date_fmt"])


(135.0, np.float64(0.0))

In [13]:
df_lite

,Datum,Zeitstempel,ObisCode,Wert,Status,date_fmt,moving_avg,quantile,threshold,high_usage,diff_percentage,ts
0,11.09.2024 10:30:00 +02:00,1726043400,01.05.2000,64.8,Normal,2024-09-11 08:30:00+00:00,77.346429,86.7,86.7,0,-5.531834,1.726043e+09
1,11.09.2024 10:15:00 +02:00,1726042500,01.05.2000,57.0,Normal,2024-09-11 08:15:00+00:00,77.369643,86.7,86.7,0,-10.174048,1.726042e+09
2,11.09.2024 10:00:00 +02:00,1726041600,01.05.2000,61.8,Normal,2024-09-11 08:00:00+00:00,77.392857,86.7,86.7,0,-7.151211,1.726042e+09
3,11.09.2024 09:45:00 +02:00,1726040700,01.05.2000,55.2,Normal,2024-09-11 07:45:00+00:00,77.393750,86.7,86.7,0,-11.444637,1.726041e+09
4,11.09.2024 09:30:00 +02:00,1726039800,01.05.2000,28.2,Normal,2024-09-11 07:30:00+00:00,77.391071,86.7,86.7,0,-39.472318,1.726040e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
57973,01.01.2023 01:15:00 +01:00,1672532100,1.5.0,128.4,Normal,2023-01-01 00:15:00+00:00,1.000000,NaN,1.0,1,16230.760000,1.672532e+09
57974,01.01.2023 01:00:00 +01:00,1672531200,1.5.0,141.6,Normal,2023-01-01 00:00:00+00:00,1.000000,NaN,1.0,1,19768.360000,1.672531e+09
57975,01.01.2023 00:45:00 +01:00,1672530300,1.5.0,125.4,Normal,2022-12-31 23:45:00+00:00,1.000000,NaN,1.0,1,15475.360000,1.672530e+09
57976,01.01.2023 00:30:00 +01:00,1672529400,1.5.0,120.6,Normal,2022-12-31 23:30:00+00:00,1.000000,NaN,1.0,1,14304.160000,1.672529e+09


In [11]:
df_lite[df_lite["user_id"] == 64].head(100)

,Datum,Zeitstempel,ObisCode,Wert,Status,date_fmt,moving_avg,quantile,threshold,high_usage,diff_percentage,ts,user_id
4,11.09.2024 00:00:00 +02:00,1726005600,1.5.0,0.12,Normal,2024-09-10 22:00:00+00:00,0.221667,0.29,0.29,0,-0.099655,1.726006e+09,64
10,10.09.2024 23:45:00 +02:00,1726004700,1.5.0,0.12,Normal,2024-09-10 21:45:00+00:00,0.221786,0.29,0.29,0,-0.099655,1.726005e+09,64
16,10.09.2024 23:30:00 +02:00,1726003800,1.5.0,0.12,Normal,2024-09-10 21:30:00+00:00,0.221726,0.29,0.29,0,-0.099655,1.726004e+09,64
22,10.09.2024 23:15:00 +02:00,1726002900,1.5.0,0.16,Normal,2024-09-10 21:15:00+00:00,0.221667,0.29,0.29,0,-0.058276,1.726003e+09,64
28,10.09.2024 23:00:00 +02:00,1726002000,1.5.0,0.20,Normal,2024-09-10 21:00:00+00:00,0.221548,0.29,0.29,0,-0.027931,1.726002e+09,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,10.09.2024 00:15:00 +02:00,1725920100,1.5.0,0.08,Normal,2024-09-09 22:15:00+00:00,0.213988,0.28,0.28,0,-0.142857,1.725920e+09,64
558,10.09.2024 00:00:00 +02:00,1725919200,1.5.0,0.16,Normal,2024-09-09 22:00:00+00:00,0.214107,0.28,0.28,0,-0.051429,1.725919e+09,64
563,09.09.2024 23:45:00 +02:00,1725918300,1.5.0,0.16,Normal,2024-09-09 21:45:00+00:00,0.213988,0.28,0.28,0,-0.051429,1.725918e+09,64
569,09.09.2024 23:30:00 +02:00,1725917400,1.5.0,0.12,Normal,2024-09-09 21:30:00+00:00,0.213929,0.28,0.28,0,-0.091429,1.725917e+09,64


In [6]:
conn = sqlite3.connect("local.db")

start = 1724937300
end = 1726004700

sql = """
Select 
    * 
from 
    user_consumption 
where 
    user_id = :user_id
    AND ts >= :start
    AND ts <= :end
"""
df_user = pd.read_sql(
    sql,
    conn,
    params={
        "user_id": 220,
        "start": start,
        "end": end,
    },
)

sql_grid = """
SELECT
    *
FROM
    grid_consumption
WHERE
    ts >= :start
    AND ts <= :end
"""
df_grid = pd.read_sql(
    sql_grid,
    conn,
    params={
        "start": start,
        "end": end,
    },
)

corr = glp.correlation_matrix_stress_level(
    df_grid,
    df_user,
    window=24 * 4,
)

conn.close()